###  1. CIFAR-10 로딩 및정규화
torchvision 을 사용하여 매우 쉽게 CIFAR-10을 로드할 수 있다.

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms

In [2]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


### 2. 합성곱 신경망(Convolution Neural Network) 정의하기
이전의 신경망 학습 자료에서 신경망을 복사한 후, (기존에 1채널 이미지만 처리하도록 정의된 것을) 3채널 이미지를 처리할 수 있도록 수정한다.

In [4]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        # First 2D convolutional layer, taking in 3 input channels (image)
        # 6 output channels, a square kernel size of 5
        self.conv1 = nn.Conv2d(3, 6, 5)
        
        # Max pooling over a (2, 2) mask
        self.pool = nn.MaxPool2d(2, 2)
        
        # Second 2D convolutional layer, taking in 6 input layers,
        # 16 output channels, a square kernel size of 5
        self.conv2 = nn.Conv2d(6, 16, 5)
        
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 5 * 5, 120) # 5*5 from image dimension
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    # x represents our data
    def forward(self, x):
        # Pass data through conv1
        # Use the rectified-linear activation function over x
        # Run max pooling over x
        x = self.pool(F.relu(self.conv1(x)))
        
        # Pass data through conv1
        # Use the rectified-linear activation function over x
        # Run max pooling over x
        x = self.pool(F.relu(self.conv2(x)))
        
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()

### 3. 손실 함수와 Optimizer 정의하기
손실함수는 Cross-Entropy loss 를 사용하고, optimizer는 모멘텀(momentum) 값을 갖는 SGD를 사용한다.

In [5]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.0001, momentum=0.9)

### 4. 신경망 학습하기
단순히 데이터를 반복해서 신경망에 입력으로 제공하고, 최적화(Optimize)만 하면 된다.

In [16]:
for epoch in range(3):   # 데이터셋을 수차례 반복

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # [inputs, labels]의 목록인 data로부터 입력을 받은 후;
        inputs, labels = data

        # 변화도(Gradient) 매개변수를 0으로 만들고
        optimizer.zero_grad()

        # 순전파 + 역전파 + 최적화를 한 후
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # 통계를 출력
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

[1,  2000] loss: 1.041
[1,  4000] loss: 1.064
[1,  6000] loss: 1.059
[1,  8000] loss: 1.030
[1, 10000] loss: 1.074
[1, 12000] loss: 1.040
[2,  2000] loss: 1.020
[2,  4000] loss: 1.022
[2,  6000] loss: 1.037
[2,  8000] loss: 1.039
[2, 10000] loss: 1.031
[2, 12000] loss: 1.030
[3,  2000] loss: 1.009
[3,  4000] loss: 1.003
[3,  6000] loss: 0.999
[3,  8000] loss: 0.995
[3, 10000] loss: 1.016
[3, 12000] loss: 1.008
Finished Training


In [17]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the 10000 test images: 61 %


----------------------------------------------------------------------------------